#Part 3:  Geospatial Join and Visualization Tutorial #
This is a tutorial to show you how to join your geospatial data to polygon geometry and visualize it in Carto.

start with a geospatial-enabled cluster with Apache Sedona installed

In [0]:
#If Sedona is not installed, like a serverless cluster, you may enable it like this.
spark.conf.set("spark.geo.st.enabled", "true")

In the upper right hand corner, select "Connect" and select "geospatial-cluster-smallest-available". It may take a few minutes to start.

## import tools 

In [0]:
%pip install folium

After installing tools, restart Python

In [0]:
dbutils.library.restartPython()

In [0]:
from pyspark.sql.functions import col, expr
import sedona
import geopandas as gpd
from sedona.sql.types import GeometryType

Go to the catalog to find the path to your data set. They are stored as Delta Tables, a variant of Parquety which are light-weight, static files on S3, which is easy to query and has version control features.

## read the table as a Spark Dataframe ##

In [0]:
sdf = spark.read.table("moo_ops_workspace.threeoneone.sr_mirror_2024")
sdf.printSchema()
sdf.display(5)

#reduce the count of files to speed it up
sdf = sdf.where(col("CREATED_DATE") > '2024-09-01')

##Parse the Lat Lon into a point##  

Carto stores everything as a WKB or Well-Known Binary. To join two geometries they need to both be in the same WKB Format. 

You may convert to WKB using ST_AsBinary(). This is called serialization. This converts it into a from that Carto can read.

You may reverse this by ST_GeomFromWKB. This is called deserialization. This converts it into a format that Databricks can read.

'ST' Either stands for Spatial Type or 'Spatial Transformation', not sure. When you cast the Latitudes and Longitudes using ST_Point, your Spark Dataframe becomes a Sedona Geospatial dataframe.

In [0]:
#turn your spark dataframe into a sedona geodataframe by creating an ST_POINT colum.
points_311 = sdf.withColumn("the_geom", expr(f"ST_AsBinary(ST_Point(LON, LAT))"))
points_311 = points_311.withColumn("geom_deserialized", expr(f"ST_AsWKT(ST_GeomFromWKB(the_geom))"))
points_311.display(5)

## Import a geospatial dataset ##
## In this step we import a Carto-Prepared geometry. ##
Carto geometry needs to have x and y max and mins for each geometry.


When you attempt to import a map into Carto, it will throw an error and ask you if you want to prepare it. A *Prepared map* adds the carto max and min bounding box to the dataframe.

For future reference, to prepare your geometry manually, you may add the following columns:

# Convert a Shapefile into a Sedona dataframe #

In [0]:
shapefile_filepath = "/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/BusinessImprovementDistrict.zip"
#if you import a shapefile, you have to specify the schema. We will use a geojson instead.
filepath = "/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/BusinessImprovementDistrict.geojson"
filepath_no_crs = "/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/BusinessImprovementDistrict_no_crs.geojson"


In [0]:
#reads the shapefile into a Pandas Geodataframe
gdf = gpd.read_file(filepath)
gdf.info()
gdf.head(2)

In [0]:
#read the shapefile as spark dataframe

#bid_map = spark.read.format("shapefile").load(shapefile_filepath)
bid_gdf = gpd.read_file(filepath_no_crs)
#convert geodataframe to pandas dataframe by deserializing the geometry
bid_gdf['geom_deserialized'] = bid_gdf['geometry'].apply(lambda x: x.wkt)
#delete the geometry, the binary will cause errors and won't load.
bid_gdf = bid_gdf.drop(columns=['geometry'])
#bid_gdf['geom_deserialized'] = bid_gdf['geometry'].apply(lambda x: str(x))
bid_gdf.head(5)
#create a spark dataframe. This is WKT, not a spatial dataframe
bid_map = spark.createDataFrame(bid_gdf)
# Add carto max and min bounding box columns
bid_map = bid_map.withColumn("__carto_xmin", expr("ST_XMin(geom_deserialized)"))
bid_map = bid_map.withColumn("__carto_xmax", expr("ST_XMax(geom_deserialized)"))
bid_map = bid_map.withColumn("__carto_ymin", expr("ST_YMin(geom_deserialized)"))
bid_map = bid_map.withColumn("__carto_ymax", expr("ST_YMax(geom_deserialized)"))
display(bid_map)

#DO NOT convert to a spatial dataframe. You will no longer be able to display it or get the schema.You also can't upload it as a table.
#bid_map = bid_map.withColumn("geometry", expr(f"ST_GeomFromWKT(geom_deserialized)"))
#this will fail
#display(bid_map)

# Optional: Partition the table #
This will help speed up queries when you have multiple workers.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", 200)
points_311 = points_311.repartition(200)
#bid_map = bid_map.repartition(200)

## Join the two maps using geospatial SQL

the point geometry will be suprerceded with the polygon geometry

In [0]:
#pass the data frames in as variables in an python f-string
points_311.createOrReplaceTempView("points_311")
spark.catalog.dropTempView("bid_map")
bid_map.createOrReplaceTempView("bid_map")
#group by all the geometry columns to keep them.
#In order to keep the map carto-prepared you need to preserve the __carto_ columns.
result_df = spark.sql(f"""SELECT
                        FIRST(__carto_xmin) AS __carto_xmin,
                        FIRST(__carto_xmax) AS __carto_xmax, 
                        FIRST(__carto_ymin) AS __carto_ymin, 
                        FIRST(__carto_ymax) AS __carto_ymax, 
                        COUNT(*) as 311_calls, 
                        bid_map.BIDID,
                        bid_map.BID,
                        FIRST(bid_map.geom_deserialized) AS geom_deserialized
                        
                      FROM points_311, bid_map
                      --ST_Contains is where the spatial join happens. It is done on the deserialized columns,
                      -- i.e. a Spark Dataframe, not a Spatial Dataframe
                      WHERE ST_Contains(bid_map.geom_deserialized, points_311.geom_deserialized)
                      AND points_311.created_date >= '2024-11-01'
                      GROUP BY  bid_map.BIDID, bid_map.BID
                      """
                      )
result_df.display()


# Save the Map#

In [0]:
%sql
DROP TABLE IF EXISTS moo_ops_workspace.threeoneone.311_calls_by_bid_test_from_databricks

In [0]:

#save result_df as a geospatial delta table in the catalog
result_df.write.format("delta").mode("overwrite").saveAsTable("moo_ops_workspace.threeoneone.311_calls_by_bid_test_from_databricks")

In [0]:
#save as a non-geospatial table
no_map_df = result_df[['311_calls', 'BID', 'BIDID']]
no_map_df.write.format('delta').mode("overwrite").saveAsTable("moo_ops_workspace.threeoneone.311_calls_by_bid_test_from_databricks_no_map")

# Visualization using Folium #

Folium is widely used in industry for map visualization.

In [0]:
#define color palate
def get_color_rgb(calls):
    if calls < 20000:
        return "rgb(255, 255, 204)"
    elif calls < 40000:
        return "rgb(255, 237, 160)"
    elif calls < 60000:
        return "rgb(254, 217, 118)"
    elif calls < 80000:
        return "rgb(254, 178, 76)"
    else:
        return "rgb(253, 141, 60)"

To visuallize the data, we need to convert from Spark to Pandas format

In [0]:
# have to get out of spark dataframe and into a geopandas geodataframe.
df = result_df.toPandas()
#load the geometry into a geopandas dataframe
df['geometry'] = df['geom_deserialized']
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf.head(5)
#apply a fill color for the data
gdf['fill_color_rgb'] = gdf['311_calls'].apply(get_color_rgb)
del gdf['geom_deserialized']
#simplify the geometry to make it smaller. Prevents the json recursion from getting too deep.
gdf['geometry'] = gdf['geometry'].simplify(0.0001)

In [0]:
import folium

In [0]:
gdf['fill_color_rgb'][:5]

Convert GeoPandas Dataframe to JSON

In [0]:
geojson = gdf.to_json()

In [0]:
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)
# folium takes color properties in the format "rgb(255, 255, 255)" or "rgba(255, 255, 255, 1.0)". It also accepts hex "#ff0000" for color
folium.GeoJson(geojson, 
               style_function=lambda feature: { 'fillColor': feature['properties']['fill_color_rgb'],
                                                'color': 'black', 
                                                'weight': 1, 
                                                'fillOpacity': 0.8
                                             }           
               ).add_to(m)
display(m)